In [1]:
!pip install pandapower

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 47.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 64.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [pandapower]5 [pandapower]


In [3]:
import math
import pandapower as pp
import pandapower.networks as nw

# --- user-configurable base parameters ---
S_base_mva = 100.0        # system base MVA used to interpret per-unit values
V_base_kv  = 230.0        # nominal voltage for all buses (kV)
f_hz = 50.0               # system frequency for converting susceptance -> capacitance
length_km_default = 1.0   # assume each line length = 1 km (so per-line B_pu is B for 1km)

# create empty network
net = pp.create_empty_network(sn_mva=S_base_mva)

# create buses (1..5)
buses = {}
for i in range(1,6):
    buses[i] = pp.create_bus(net, vn_kv=V_base_kv, name=f"Bus {i}")

# external grid (slack) at bus 1 with voltage 1.06 pu
pp.create_ext_grid(net, bus=buses[1], vm_pu=1.06, name="Slack Bus 1")

# create loads (from table)
loads = {
    2: (20.0, 10.0),
    3: (45.0, 15.0),
    4: (40.0, 5.0),
    5: (60.0, 10.0)
}
for bus, (p,q) in loads.items():
    pp.create_load(net, bus=buses[bus], p_mw=p, q_mvar=q, name=f"Load at bus {bus}")

# create generator at bus 2 (modeled as sgen with fixed P and Q)
pp.create_sgen(net, bus=buses[2], p_mw=40.0, q_mvar=30.0, name="Gen at bus 2 (40+ j30)")

# line data from table (R_pu, X_pu, B_pu total per line)
line_data = [
    (1, 2, 0.02, 0.06, 0.025),
    (1, 3, 0.08, 0.24, 0.025),
    (2, 3, 0.06, 0.25, 0.020),
    (2, 4, 0.06, 0.18, 0.020),
    (2, 5, 0.04, 0.12, 0.015),
    (3, 4, 0.01, 0.03, 0.010),
    (4, 5, 0.08, 0.24, 0.025),
]

# conversion factors
Z_base = (V_base_kv*1e3)**2 / (S_base_mva*1e6)   # ohm
for (p,q,r_pu,x_pu,b_pu) in line_data:
    r_ohm_per_km = r_pu * Z_base / length_km_default
    x_ohm_per_km = x_pu * Z_base / length_km_default
    # convert shunt susceptance (pu) to capacitance nF/km for pandapower
    # B_s (S) = B_pu * S_base / V_base^2
    B_siemens = b_pu * (S_base_mva*1e6) / (V_base_kv*1e3)**2
    # capacitance per km: C = B / (2*pi*f)
    c_f_per_km = B_siemens / (2*math.pi*f_hz)
    c_nf_per_km = c_f_per_km * 1e9
    pp.create_line_from_parameters(net,
                                   from_bus=buses[p],
                                   to_bus=buses[q],
                                   length_km=length_km_default,
                                   r_ohm_per_km=r_ohm_per_km,
                                   x_ohm_per_km=x_ohm_per_km,
                                   c_nf_per_km=c_nf_per_km,
                                   max_i_ka=1.0,
                                   name=f"{p}-{q} line")

# Run powerflow with v_debug=True so per-iteration info is kept
pp.runpp(net, algorithm="nr", calculate_voltage_angles=True, v_debug=True)

# Helper to extract iterations
def get_runpp_iterations(net):
    ppc = net.get("_ppc", None)
    if not ppc:
        return None
    # 1) direct top-level field (some versions/backend expose this)
    if isinstance(ppc, dict) and "iterations" in ppc:
        return ppc["iterations"]
    # 2) internal dict (backend-specific)
    internal = ppc.get("internal", {}) if isinstance(ppc, dict) else {}
    if isinstance(internal, dict) and "iterations" in internal:
        return internal["iterations"]
    # 3) fallback: count stored per-iteration voltage vectors (Vm_it or Va_it)
    vm_it = internal.get("Vm_it")
    va_it = internal.get("Va_it")
    if vm_it is not None:
        try:
            return len(vm_it)
        except Exception:
            pass
    if va_it is not None:
        try:
            return len(va_it)
        except Exception:
            pass
    # 4) nothing found
    return None

iters = get_runpp_iterations(net)
print("Newton-Raphson iterations:", iters)

# Optional debug: if iters is None, print keys to inspect where pandapower stored data
# Uncomment if you want to inspect the structure
# import pprint
# pprint.pprint({ 'ppc_keys': list(net.get("_ppc",{}).keys()),
#                 'internal_keys': list(net.get("_ppc",{}).get("internal",{}).keys()) })

# show key results
print("\nBus voltages (pu):")
for i in range(1,6):
    b_idx = buses[i]
    vm = net.res_bus.vm_pu.at[b_idx]
    va = net.res_bus.va_degree.at[b_idx]
    print(f" Bus {i}: V = {vm:.4f} pu  angle = {va:.3f} deg")

print("\nExternal grid (slack) power injection (positive = supplying into net):")
print(net.res_ext_grid[["p_mw","q_mvar"]])

print("\nGenerators (sgen) results (these are fixed setpoints for this model):")
print(net.res_sgen[["p_mw","q_mvar"]])

print("\nLoads (results):")
print(net.res_load[["p_mw","q_mvar"]])

# compute power factor for each generator and load (pf = P / sqrt(P^2 + Q^2))
print("\nPower factors (per device):")
for idx, row in net.res_sgen.iterrows():
    P = row.p_mw
    Q = row.q_mvar
    S = math.hypot(P, Q)
    pf = P / S if S != 0 else 0.0
    print(f" SGEN {net.sgen.at[idx,'name']}: P={P:.3f} MW, Q={Q:.3f} MVar, PF={pf:.4f} (lagging if Q>0)")

for idx, row in net.res_load.iterrows():
    P = row.p_mw
    Q = row.q_mvar
    S = math.hypot(P, Q)
    pf = P / S if S != 0 else 0.0
    print(f" LOAD {net.load.at[idx,'name']}: P={P:.3f} MW, Q={Q:.3f} MVar, PF={pf:.4f} (lagging if Q>0)")

# system-level PF: consider total generation injection (sgen + ext_grid)
total_P_gen = net.res_sgen.p_mw.sum() + net.res_ext_grid.p_mw.sum()
total_Q_gen = net.res_sgen.q_mvar.sum() + net.res_ext_grid.q_mvar.sum()
S_gen = math.hypot(total_P_gen, total_Q_gen)
pf_gen = total_P_gen / S_gen if S_gen != 0 else 0.0
print(f"\nSystem-level generation: P={total_P_gen:.3f} MW, Q={total_Q_gen:.3f} MVar, PF={pf_gen:.4f} (lagging if Q>0)")

# system-level load PF (sum of loads)
total_P_load = net.res_load.p_mw.sum()
total_Q_load = net.res_load.q_mvar.sum()
S_load = math.hypot(total_P_load, total_Q_load)
pf_load = total_P_load / S_load if S_load != 0 else 0.0
print(f"System-level loads: P={total_P_load:.3f} MW, Q={total_Q_load:.3f} MVar, PF={pf_load:.4f} (lagging if Q>0)")

# optionally display net table for inspection
print("\nNet summary tables available: net.bus, net.line, net.load, net.sgen, net.res_bus, net.res_line")


numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


Newton-Raphson iterations: 3

Bus voltages (pu):
 Bus 1: V = 1.0600 pu  angle = 0.000 deg
 Bus 2: V = 1.0420 pu  angle = -2.666 deg
 Bus 3: V = 1.0148 pu  angle = -5.114 deg
 Bus 4: V = 1.0141 pu  angle = -5.390 deg
 Bus 5: V = 1.0092 pu  angle = -6.081 deg

External grid (slack) power injection (positive = supplying into net):
         p_mw    q_mvar
0  129.701057  9.536521

Generators (sgen) results (these are fixed setpoints for this model):
   p_mw  q_mvar
0  40.0    30.0

Loads (results):
   p_mw  q_mvar
0  20.0    10.0
1  45.0    15.0
2  40.0     5.0
3  60.0    10.0

Power factors (per device):
 SGEN Gen at bus 2 (40+ j30): P=40.000 MW, Q=30.000 MVar, PF=0.8000 (lagging if Q>0)
 LOAD Load at bus 2: P=20.000 MW, Q=10.000 MVar, PF=0.8944 (lagging if Q>0)
 LOAD Load at bus 3: P=45.000 MW, Q=15.000 MVar, PF=0.9487 (lagging if Q>0)
 LOAD Load at bus 4: P=40.000 MW, Q=5.000 MVar, PF=0.9923 (lagging if Q>0)
 LOAD Load at bus 5: P=60.000 MW, Q=10.000 MVar, PF=0.9864 (lagging if Q>0)

Sys

In [6]:
import pandapower as pp
print(pp.__version__)


3.1.2
